In [ ]:
%run dataImport.ipynb

In [ ]:
path = "/home/fabienne/Documents/MA/Data/Dataset_MA.csv" # because of data privacy reasons, I was not allowed to publish the dataset
df = time_series(path)

In [ ]:
df_t = df
df_t['Datetime'] = pd.to_datetime(df['Datetime'])
df_t = df_t.set_index('Datetime')

In [ ]:
f = df_t['Aktueller_Strombezug_kW'].first_valid_index()
s = df_t['Aktueller_Strombezug_kW'].last_valid_index()

In [ ]:
print(len(df_t))
print(df.columns)

In [ ]:
print(df_t["Aktuelle_BHKW_Leistung_kW"].head())

## Frequency of variables

# Aggregate Dataset


In [ ]:
# fill the values for the variables that are just made once in a day
for i in ["Mitarbeiter_Geplant", "Mitarbeiter_Summe", "Produktion_Geplant"]:
    df_t[i] = df_t.groupby(df_t.index.date)[i].fillna(method='ffill')

In [ ]:
# aggregate data
ag_df = df_t.resample('6H').mean()
print(len(ag_df))
# do majority voting for dummy variables(can just be 0 or 1)
tmp = df_t[["BHKW1_Betriebsstatus" ,"BHKW2_Betriebsstatus"]]
tmp = df_t.resample('6H').apply(lambda x: 0 if (x == 0).sum() > (x == 1).sum() else 1)
print(len(tmp))

In [ ]:
# auf ganze zahlen runden bei mitarbeitern
for i in ["Mitarbeiter_Geplant", "Mitarbeiter_Summe"]:
    ag_df[i] = ag_df[i].round()

In [ ]:
# extract features
fst = tmp["BHKW1_Betriebsstatus"].to_numpy()
scd = tmp["BHKW2_Betriebsstatus"].to_numpy()
# replace in dataset
ag_df["BHKW1_Betriebsstatus"] = fst
ag_df["BHKW2_Betriebsstatus"] = scd

In [ ]:
kw = ag_df["Aktueller_Strombezug_kW"].mean()
kw.round(3)
#print(ag_df.head())
#print(ag_df.head(-1))

# Preprocessing

In [ ]:
var = ag_df.columns.tolist()
pos = var[:]
norm = var[:]
pos.remove('Aussentemperatur')
norm.remove('Aktueller_Strombezug_kW')

In [ ]:
# function fo just positive values
def pos_values(df,col):
    df.loc[df[col] < 0, col] = 0

for i in pos:
    pos_values(ag_df, i)

In [ ]:
# check of the dummy variables are just 0 and 1 
for i in ['BHKW1_Betriebsstatus','BHKW2_Betriebsstatus']:
    column_check = ag_df[i].isin([0.0, 1.0, np.nan]).all()
    if column_check:
        print(f"Values in {i} are only 0 and 1 or NAN.")
    else:
        print(f"Values in {i} are not only 0 and 1 or NAN.")

In [ ]:
unique_values = ag_df['BHKW2_Betriebsstatus'].value_counts()

print(unique_values)


In [ ]:
def normalize(df, var):
    x = (df[var]-df[var].min())/(df[var].max()-df[var].min())
    df[var] = x

# normalize all rows
for i in norm:
    normalize(ag_df, i)

In [ ]:
# check if the y variable has some missing values
def check_nan(df):
    has_nan_values = df.isna().any().any()
    if has_nan_values:
        nan_counts = df.isna().sum()
        return nan_counts
    else:
        return {}

In [ ]:
# use linear interpolation to fill the gaps
def linear_interpolation(df,col):
    df[col] = df[col].interpolate()
    
for i in var:
    linear_interpolation(ag_df, i)

In [ ]:
# if no interpolation possible use ffill an bfill methods to fill rest of nan values
def fill_forward(df,col):
    df[col] = df[col].ffill()
    
def fill_backwards(df,col):
    df[col] = df[col].bfill()

In [ ]:
# first fill forward
for i in var:
    fill_forward(ag_df,i)
# fill backwards
for i in var:
    fill_backwards(ag_df, i)

In [ ]:
# remove all the columns that still have nan values 
# cause in this columns there are only nan values 
dic = check_nan(ag_df)
lst_keys = [key for key, value in dic.items() if value != 0]
df_pre = ag_df.drop(columns = lst_keys)
check = check_nan(df_pre)
print(check)
#print(df_pre.head())

In [ ]:
konstante_spalten = df_pre.columns[df_pre.nunique() == 1]

print(konstante_spalten)

In [ ]:
# drop columns
df_ts = df_pre[["Mitarbeiter_Summe","Aktuelle_BHKW_Leistung_kW","Aktueller_Strombezug_kW", "Aussentemperatur","Wirkleistung_Verbrauch", 
                  'BHKW1_Betriebsstatus','BHKW2_Betriebsstatus']]


# Save preprocessed dataset

In [ ]:
df_ts.to_csv("time_series.csv", index =True)

In [ ]:
# make a train and a test dataset
n = len(df_ts)

# Split the DataFrame into two parts: first n-360 observations and last 360 observations
train_df = df_ts.head(n - 360)
test_df = df_ts.tail(360)

train_df.to_csv("train_df.csv", index =True)
test_df.to_csv("test_df.csv", index =True)